In [1]:
from model import create_model

nn4_small2 = create_model()

Using TensorFlow backend.


In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

in_a = Input(shape=(96, 96, 3))
in_p = Input(shape=(96, 96, 3))
in_n = Input(shape=(96, 96, 3))


emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)

In [4]:
from data import triplet_generator

generator = triplet_generator() 

nn4_small2_train.compile(loss=None, optimizer='adam')
nn4_small2_train.fit_generator(generator, epochs=10, steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 26s 263ms/step - loss: 0.8092
Epoch 2/10
100/100 [==============================] - 7s 66ms/step - loss: 0.8055
Epoch 3/10
100/100 [==============================] - 7s 70ms/step - loss: 0.8008
Epoch 4/10
100/100 [==============================] - 6s 63ms/step - loss: 0.8003
Epoch 5/10
100/100 [==============================] - 7s 66ms/step - loss: 0.8002
Epoch 6/10
 85/100 [========================>.....] - ETA: 0s - loss: 0.8001

KeyboardInterrupt: 